# CH07-01: Transformer 架構概覽

**課程目標:**
- 理解 Transformer 的誕生背景與革命性意義
- 掌握 Transformer 的整體架構與核心組件
- 了解 Self-Attention 機制的直觀概念
- 認識 Transformer 在 NLP 領域的重要應用

**學習時間:** 約 90 分鐘

**前置知識:**
- 基礎神經網路 (MLP)
- 序列模型 (RNN/LSTM)
- 詞向量 (Word Embeddings)

---

## 📚 目錄

1. [為什麼需要 Transformer?](#1)
2. [Transformer 架構全貌](#2)
3. [核心組件介紹](#3)
4. [Attention 機制直觀理解](#4)
5. [Transformer 的優勢與應用](#5)
6. [實戰練習](#6)

---

In [ ]:
# 環境設定與套件導入
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定顯示風格
sns.set_style('whitegrid')

print("✅ 環境設定完成")
print(f"NumPy 版本: {np.__version__}")

<a id="1"></a>
## 1. 為什麼需要 Transformer?

### 1.1 傳統序列模型的挑戰

在 Transformer 出現之前,NLP 領域主要使用 **RNN** 和 **LSTM** 處理序列數據。

#### RNN/LSTM 的三大限制:

1. **無法並行化 (Sequential Processing)**
   - 必須按順序逐步處理,無法利用 GPU 並行加速
   - 訓練速度慢,難以擴展到大型數據集

2. **長距離依賴問題 (Long-Range Dependencies)**
   - 儘管 LSTM 使用門控機制,但處理超長序列仍力不從心
   - 梯度消失/爆炸問題

3. **固定上下文窗口 (Fixed Context Window)**
   - 隱藏狀態有限,難以捕捉全局訊息
   - 容易遺忘早期重要訊息

---

### 1.2 Transformer 的革命性突破

**論文:** *Attention is All You Need* (Vaswani et al., 2017)

**核心思想:** 完全拋棄循環結構,純粹使用 **Self-Attention 機制**!

#### Transformer 的三大優勢:

1. ✅ **完全並行化** - 所有位置同時計算,訓練速度快 10-100 倍
2. ✅ **全局視野** - 每個詞可以直接關注序列中的任何其他詞
3. ✅ **可擴展性** - 可輕鬆擴展到數十億參數 (GPT-3: 175B)

---

In [ ]:
# 視覺化: RNN vs Transformer 訓練速度對比
sequence_lengths = [50, 100, 200, 500, 1000]
rnn_time = [1, 4, 16, 100, 400]  # 模擬訓練時間 (秒)
transformer_time = [2, 3, 4, 6, 10]  # 並行化優勢

plt.figure(figsize=(10, 6))
plt.plot(sequence_lengths, rnn_time, 'o-', label='RNN/LSTM (Sequential)', linewidth=2, markersize=8)
plt.plot(sequence_lengths, transformer_time, 's-', label='Transformer (Parallel)', linewidth=2, markersize=8)
plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('Training Time (seconds)', fontsize=12)
plt.title('RNN vs Transformer 訓練速度對比 (模擬)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

print("📊 觀察:")
print("  - RNN: 序列越長,訓練時間指數增長")
print("  - Transformer: 序列長度影響小,可並行處理")

<a id="2"></a>
## 2. Transformer 架構全貌

### 2.1 整體架構

Transformer 採用經典的 **Encoder-Decoder** 結構:

```
輸入序列 (Input)
    ↓
┌─────────────────────────┐
│   Encoder (編碼器)        │  ← 理解輸入語義
│   - N 層堆疊              │
│   - Self-Attention       │
│   - Feed Forward         │
└─────────────────────────┘
    ↓ (Context Vector)
┌─────────────────────────┐
│   Decoder (解碼器)        │  ← 生成輸出序列
│   - N 層堆疊              │
│   - Masked Self-Attention│
│   - Cross-Attention      │
│   - Feed Forward         │
└─────────────────────────┘
    ↓
輸出序列 (Output)
```

---

### 2.2 核心組件解析

#### 2.2.1 Encoder 層結構

每個 Encoder 層包含兩個子層:

1. **Multi-Head Self-Attention**
   - 讓每個詞關注序列中的所有詞
   - 多頭機制捕捉不同關係 (語法、語義等)

2. **Position-wise Feed-Forward Network**
   - 兩層全連接網路
   - 對每個位置獨立處理

**每個子層後都有:**
- Residual Connection (殘差連接)
- Layer Normalization (層歸一化)

---

#### 2.2.2 Decoder 層結構

每個 Decoder 層包含三個子層:

1. **Masked Multi-Head Self-Attention**
   - 防止看到未來的詞 (自回歸生成)

2. **Cross-Attention (Encoder-Decoder Attention)**
   - Decoder 關注 Encoder 的輸出
   - 連接輸入與輸出的橋樑

3. **Position-wise Feed-Forward Network**
   - 與 Encoder 相同

---

In [ ]:
# 視覺化: Transformer 架構層級結構
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 12)
ax.axis('off')

# Encoder 區塊
encoder_rect = mpatches.Rectangle((0.5, 6), 4, 5, 
                                   linewidth=2, edgecolor='blue', facecolor='lightblue', alpha=0.3)
ax.add_patch(encoder_rect)
ax.text(2.5, 10.5, 'ENCODER', ha='center', fontsize=16, fontweight='bold', color='darkblue')

# Encoder 子層
ax.text(2.5, 9.5, 'Multi-Head\nSelf-Attention', ha='center', fontsize=11, 
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='blue'))
ax.text(2.5, 8, 'Add & Norm', ha='center', fontsize=9, style='italic')
ax.text(2.5, 7.2, 'Feed Forward\nNetwork', ha='center', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='blue'))
ax.text(2.5, 6.4, 'Add & Norm', ha='center', fontsize=9, style='italic')

# Decoder 區塊
decoder_rect = mpatches.Rectangle((5.5, 6), 4, 5, 
                                   linewidth=2, edgecolor='green', facecolor='lightgreen', alpha=0.3)
ax.add_patch(decoder_rect)
ax.text(7.5, 10.5, 'DECODER', ha='center', fontsize=16, fontweight='bold', color='darkgreen')

# Decoder 子層
ax.text(7.5, 9.7, 'Masked\nSelf-Attention', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='green'))
ax.text(7.5, 9, 'Add & Norm', ha='center', fontsize=9, style='italic')
ax.text(7.5, 8.2, 'Cross-Attention', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='green'))
ax.text(7.5, 7.5, 'Add & Norm', ha='center', fontsize=9, style='italic')
ax.text(7.5, 6.7, 'Feed Forward', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='white', edgecolor='green'))

# 輸入與輸出
ax.text(2.5, 5.3, 'Input Embedding\n+\nPositional Encoding', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax.text(7.5, 5.3, 'Output Embedding\n+\nPositional Encoding', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightyellow'))

# 箭頭
ax.arrow(2.5, 5.8, 0, 0.15, head_width=0.2, head_length=0.1, fc='black', ec='black')
ax.arrow(7.5, 5.8, 0, 0.15, head_width=0.2, head_length=0.1, fc='black', ec='black')
ax.arrow(4.5, 8.5, 0.9, 0, head_width=0.15, head_length=0.1, fc='red', ec='red', linestyle='--')
ax.text(5, 8.8, 'Context', ha='center', fontsize=9, color='red')

# 標題
ax.text(5, 11.5, 'Transformer 架構全貌', ha='center', fontsize=18, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🔍 架構重點:")
print("  1. Encoder: 將輸入編碼為語義表示")
print("  2. Decoder: 基於編碼生成輸出序列")
print("  3. Cross-Attention: Encoder 與 Decoder 的橋樑")

<a id="3"></a>
## 3. 核心組件深入

### 3.1 位置編碼 (Positional Encoding)

**問題:** Attention 機制是**無序的** (permutation-invariant),無法區分詞的順序!

**解決方案:** 在詞向量中加入位置訊息

#### 正弦位置編碼公式:

$$
\text{PE}_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

$$
\text{PE}_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

其中:
- `pos`: 詞在序列中的位置 (0, 1, 2, ...)
- `i`: 維度索引 (0, 1, 2, ..., d_model/2)
- `d_model`: 詞向量維度 (論文中為 512)

**優點:**
- 不同位置有獨特的編碼
- 相對位置關係可學習
- 可處理任意長度序列

---

In [ ]:
# 實作並視覺化位置編碼
def get_positional_encoding(max_seq_len, d_model):
    """
    生成正弦位置編碼
    
    Args:
        max_seq_len: 最大序列長度
        d_model: 詞向量維度
        
    Returns:
        pos_encoding: shape (max_seq_len, d_model)
    """
    pos_encoding = np.zeros((max_seq_len, d_model))
    
    for pos in range(max_seq_len):
        for i in range(0, d_model, 2):
            # Sin for even indices
            pos_encoding[pos, i] = np.sin(pos / (10000 ** (i / d_model)))
            
            # Cos for odd indices
            if i + 1 < d_model:
                pos_encoding[pos, i + 1] = np.cos(pos / (10000 ** (i / d_model)))
    
    return pos_encoding

# 生成位置編碼
max_len = 100
d_model = 512
pos_enc = get_positional_encoding(max_len, d_model)

print(f"位置編碼矩陣形狀: {pos_enc.shape}")
print(f"前 3 個位置的前 8 維編碼:\n{pos_enc[:3, :8]}")

# 視覺化位置編碼
plt.figure(figsize=(12, 6))
plt.imshow(pos_enc[:50, :128].T, cmap='RdBu', aspect='auto')
plt.colorbar(label='Encoding Value')
plt.xlabel('Position in Sequence', fontsize=12)
plt.ylabel('Embedding Dimension', fontsize=12)
plt.title('正弦位置編碼視覺化 (前 50 位置, 前 128 維度)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎨 觀察:")
print("  - 每個位置有獨特的編碼模式")
print("  - 低維度變化快 (高頻),高維度變化慢 (低頻)")
print("  - 相鄰位置編碼相似,但不完全相同")

### 3.2 殘差連接與層歸一化

#### 3.2.1 殘差連接 (Residual Connection)

$$
\text{Output} = \text{SubLayer}(x) + x
$$

**作用:**
- 緩解梯度消失問題
- 讓訊息更容易傳遞
- 加速訓練收斂

---

#### 3.2.2 層歸一化 (Layer Normalization)

$$
\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

**作用:**
- 穩定訓練過程
- 加速收斂
- 減少內部協變量偏移

---

<a id="4"></a>
## 4. Attention 機制直觀理解

### 4.1 什麼是 Attention?

**核心思想:** 讓模型"關注"最相關的部分

#### 人類閱讀的例子:

句子: *"The **animal** didn't cross the street because **it** was too tired."*

問題: "it" 指的是什麼?

人類會自動關注:
- "it" ← **高度關注** ← "animal"
- "it" ← 低度關注 ← "street"

**這就是 Attention!**

---

### 4.2 Self-Attention 機制

**三個關鍵矩陣:**

1. **Query (Q)**: "我想查詢什麼?"
2. **Key (K)**: "我是什麼?"
3. **Value (V)**: "我有什麼訊息?"

#### Attention 計算公式:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$

**步驟解析:**
1. 計算 Query 與 Key 的相似度: $QK^T$
2. 縮放 (避免數值過大): $\frac{QK^T}{\sqrt{d_k}}$
3. Softmax 歸一化為概率分布
4. 加權組合 Value: $\text{softmax}(...) \cdot V$

---

In [ ]:
# 簡單的 Self-Attention 實作示例
def simple_self_attention(X, d_k):
    """
    簡化版 Self-Attention 機制
    
    Args:
        X: 輸入矩陣 (seq_len, d_model)
        d_k: Key 維度 (用於縮放)
        
    Returns:
        output: Attention 輸出
        attention_weights: Attention 權重矩陣
    """
    # 簡化: 直接使用 X 作為 Q, K, V (實際應用需投影矩陣)
    Q = K = V = X
    
    # Step 1: 計算相似度 (Dot Product)
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    
    # Step 2: Softmax 歸一化
    attention_weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)
    
    # Step 3: 加權組合 Value
    output = np.dot(attention_weights, V)
    
    return output, attention_weights

# 測試範例
np.random.seed(42)
seq_len = 5
d_model = 8
d_k = d_model

# 模擬輸入序列
X = np.random.randn(seq_len, d_model)

# 計算 Attention
output, attn_weights = simple_self_attention(X, d_k)

print(f"輸入形狀: {X.shape}")
print(f"輸出形狀: {output.shape}")
print(f"Attention 權重形狀: {attn_weights.shape}")
print(f"\nAttention 權重矩陣:\n{attn_weights}")

# 視覺化 Attention 權重
plt.figure(figsize=(8, 6))
sns.heatmap(attn_weights, annot=True, fmt='.2f', cmap='YlOrRd', 
            xticklabels=[f'位置 {i+1}' for i in range(seq_len)],
            yticklabels=[f'位置 {i+1}' for i in range(seq_len)],
            cbar_kws={'label': 'Attention Score'})
plt.xlabel('Key (被關注的位置)', fontsize=12)
plt.ylabel('Query (查詢位置)', fontsize=12)
plt.title('Self-Attention 權重矩陣視覺化', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 解讀:")
print("  - 每一行總和為 1 (Softmax 特性)")
print("  - 顏色越深表示關注度越高")
print("  - 對角線值較高: 自己關注自己")

### 4.3 Multi-Head Attention

**核心思想:** 使用多個 Attention "頭",各自學習不同關係

#### 為什麼需要多頭?

一個句子中可能有多種關係:
- **語法關係:** 主詞-動詞-受詞
- **語義關係:** 近義詞、反義詞
- **指代關係:** 代名詞指涉

**多頭讓模型同時學習這些不同關係!**

#### 多頭 Attention 公式:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h) W^O
$$

其中每個頭:

$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

**論文設定:** 8 個頭,每個頭維度 64 (總維度 512)

---

<a id="5"></a>
## 5. Transformer 的優勢與應用

### 5.1 Transformer 三大優勢

| 特性 | RNN/LSTM | Transformer |
|------|----------|-------------|
| **並行化** | ❌ 只能串行處理 | ✅ 完全並行 |
| **長距離依賴** | ⚠️ 受限於隱藏狀態 | ✅ 直接連接所有位置 |
| **訓練速度** | 🐌 慢 | 🚀 快 10-100 倍 |
| **可擴展性** | ⚠️ 難以擴展 | ✅ 可擴展到數十億參數 |
| **全局視野** | ❌ 有限上下文窗口 | ✅ 全序列視野 |

---

### 5.2 Transformer 三大模型家族

#### 5.2.1 Encoder-Only (BERT 系列)

**代表:** BERT, RoBERTa, ALBERT

**特點:**
- 雙向 Attention (可看到整個句子)
- 適合理解任務

**應用:**
- 文本分類
- 命名實體識別 (NER)
- 問答系統

---

#### 5.2.2 Decoder-Only (GPT 系列)

**代表:** GPT-2, GPT-3, GPT-4, Claude

**特點:**
- 單向 Attention (只看前文)
- 自回歸生成

**應用:**
- 文本生成
- 對話系統
- 程式碼生成

---

#### 5.2.3 Encoder-Decoder (T5 系列)

**代表:** T5, BART, mT5

**特點:**
- 完整 Transformer 架構
- 輸入→輸出轉換

**應用:**
- 機器翻譯
- 文本摘要
- 問答生成

---

### 5.3 Transformer 的影響力

**2017-2024 NLP 發展史:**

```
2017 → Transformer 論文發表 ("Attention is All You Need")
2018 → BERT 橫空出世 (刷新 11 項 NLP 紀錄)
2019 → GPT-2 展示強大生成能力
2020 → GPT-3 (175B 參數) 驚豔世界
2022 → ChatGPT 引爆 AI 熱潮
2023 → GPT-4, Claude, LLaMA 百花齊放
2024 → Transformer 應用延伸至視覺、多模態
```

**結論:** Transformer 徹底改變了 AI 領域!

---

<a id="6"></a>
## 6. 實戰練習

### 練習 1: 位置編碼實驗

修改位置編碼函數,嘗試不同的編碼方式,觀察視覺化差異。

In [ ]:
# 練習 1: 實作可學習的位置編碼
def learned_positional_encoding(max_seq_len, d_model):
    """
    可學習的位置編碼 (隨機初始化,實際訓練時會更新)
    """
    # TODO: 使用隨機初始化替代正弦函數
    return np.random.randn(max_seq_len, d_model) * 0.1

# 比較兩種編碼
sinusoidal_pe = get_positional_encoding(50, 128)
learned_pe = learned_positional_encoding(50, 128)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 正弦編碼
im1 = axes[0].imshow(sinusoidal_pe.T, cmap='RdBu', aspect='auto')
axes[0].set_title('正弦位置編碼 (固定)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Position')
axes[0].set_ylabel('Dimension')
plt.colorbar(im1, ax=axes[0])

# 可學習編碼
im2 = axes[1].imshow(learned_pe.T, cmap='RdBu', aspect='auto')
axes[1].set_title('可學習位置編碼 (初始化)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Position')
axes[1].set_ylabel('Dimension')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

print("💡 思考:")
print("  1. 兩種編碼的視覺化有什麼差異?")
print("  2. 哪一種更適合處理長序列?")
print("  3. GPT 系列使用哪一種編碼?")

### 練習 2: Attention 權重分析

分析一個實際句子的 Attention 模式。

In [ ]:
# 練習 2: 分析真實句子的 Attention
sentence = "The cat sat on the mat".split()
seq_len = len(sentence)

# 模擬 Attention 權重 (實際需要訓練好的模型)
np.random.seed(123)
attn = np.random.rand(seq_len, seq_len)
attn = attn / attn.sum(axis=1, keepdims=True)  # 歸一化

# 視覺化
plt.figure(figsize=(10, 8))
sns.heatmap(attn, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=sentence,
            yticklabels=sentence,
            cbar_kws={'label': 'Attention Score'})
plt.xlabel('被關注的詞 (Key)', fontsize=12)
plt.ylabel('查詢詞 (Query)', fontsize=12)
plt.title('句子: "The cat sat on the mat" 的 Attention 模式', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔍 分析:")
print(f"  - 'cat' 最關注哪些詞? {sentence[np.argmax(attn[1])]}")
print(f"  - 'sat' 最關注哪些詞? {sentence[np.argmax(attn[2])]}")
print("\n💭 思考: 實際訓練的模型會學到什麼樣的 Attention 模式?")

---

## 📚 本課總結

### 核心要點回顧:

1. **Transformer 革命性創新:**
   - 完全拋棄 RNN,純粹使用 Self-Attention
   - 並行化訓練,速度提升 10-100 倍
   - 可擴展到數十億參數的大型語言模型

2. **架構組件:**
   - Encoder-Decoder 雙塔結構
   - Multi-Head Self-Attention 核心機制
   - 位置編碼解決詞序問題
   - 殘差連接與層歸一化穩定訓練

3. **Attention 機制:**
   - Query, Key, Value 三要素
   - Softmax 歸一化權重
   - 多頭學習不同關係

4. **三大模型家族:**
   - Encoder-Only (BERT): 理解任務
   - Decoder-Only (GPT): 生成任務
   - Encoder-Decoder (T5): 轉換任務

---

## 🎯 下節預告

**CH07-02: 嵌入層 (Embeddings)**

我們將深入探討:
- Token Embeddings 詞嵌入
- 位置編碼的數學原理
- Word2Vec, GloVe vs Transformer Embeddings
- 實戰: 訓練自己的詞向量

---

## 📖 延伸閱讀

1. **論文:**
   - [Attention is All You Need](https://arxiv.org/abs/1706.03762) (Vaswani et al., 2017)
   - [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/) (Jay Alammar)

2. **教學資源:**
   - [Stanford CS224N Lecture on Transformers](https://web.stanford.edu/class/cs224n/)
   - [Hugging Face Transformers Course](https://huggingface.co/learn/nlp-course)

3. **實作教學:**
   - [Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html)
   - [The Transformer Family 2.0](https://lilianweng.github.io/posts/2023-01-27-the-transformer-family-v2/)

---

### 🙋 問題討論

有任何問題嗎?歡迎在討論區提問!

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)